In [1]:
import sys
import os
import pandas as pd
import numpy as np
from random import randint
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch import optim as optim
import torch.autograd as autograd
import importlib
import Extract_data
importlib.reload(Extract_data)
import Inference
importlib.reload(Inference)
import matplotlib.pyplot as plt
import torch.nn.functional as F
import random
import sys
from os import walk
import random
import FCNN_serial
importlib.reload(FCNN_serial)
from scipy import signal
import prototypical_loss
importlib.reload(prototypical_loss)

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [5]:
device=torch.device("cpu")

In [ ]:
'''
Evaluate
'''

In [40]:
n_support=6
n_query=5
n_samples=n_support+n_query
n_samples

11

In [16]:
data_df,seen_classes,unseen_classes,sr=Extract_data.prepare_UTWENTE_data()
participants=data_df['participant'].unique()

In [41]:
data_df,seen_classes,unseen_classes,sr=Extract_data.prepare_UTWENTE_data()
participants=data_df['participant'].unique()

source_data,target_data=Extract_data.get_seen_unseen_data(data_df,seen_classes,unseen_classes,['1'],['1'])

bs=60
num_features=128
in_channels=data_df['data'].iloc[0].shape[0]
num_classes=len(np.unique(source_data['label']))
lr=0.001
discount=0.6
segment_len_s=1
window_len=sr*segment_len_s
overlap=int(window_len*0.5)
LAMBDA=0.0001
num_selected=5
kernel1_size=2
kernel2_size=1
num_skip=20

model=FCNN_serial.HARmodel(num_classes=num_classes,in_channels=in_channels,
                                    num_features=num_features,
                                    kernel1_size=kernel1_size,
                                    kernel2_size=kernel2_size)
model=model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=lr)

windowed_df=Extract_data.get_windowed_df(data_df,window_len,overlap)
participant1=participant2=participants[0]
source_df,target_df=Extract_data.get_seen_unseen_data(windowed_df,seen_classes,unseen_classes,[participant1],[participant2])

min_num=np.min(source_df.groupby('label').count()['data'].values)
train_df,test_df=Extract_data.divide_data_samples(source_df,num_samples=min_num-3)

selected_df,other_df=Extract_data.select_random_classes_data(train_df,num_selected)

selected_batch,_=Extract_data.divide_data_samples(selected_df,num_samples=n_samples)
selected_data=torch.from_numpy(np.array([d for d in selected_batch['data']]))
selected_labels=torch.from_numpy(np.array([d for d in selected_batch['label']]))
selected_data=selected_data.float()  
selected_data=selected_data.to(device)
selected_labels=selected_labels.to(device)

selected_embedding,selected_pred=model(selected_data)

In [75]:
loss, acc=loss_fn(selected_embedding,target=selected_labels,n_support=n_support)

In [73]:
selected_labels.shape

torch.Size([55])

In [31]:
selected_df.groupby('label').count()

,activity_name,data,participant
label,,,
0,1787,1787,1787
1,1787,1787,1787
2,1787,1787,1787
4,1787,1787,1787
6,1787,1787,1787


In [12]:
selected_df['activity_name'].unique(),other_df['activity_name'].unique()

(array(['walk', 'upstairs', 'type', 'drink', 'talk'], dtype=object),
 array(['stand', 'smoke', 'eat'], dtype=object))

In [63]:
def euclidean_dist(x, y):
    '''
    Compute euclidean distance between two tensors
    '''
    # x: N x D
    # y: M x D
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)
    if d != y.size(1):
        raise Exception

    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)

    return torch.pow(x - y, 2).sum(2)

In [54]:
target=selected_labels
input=selected_embedding

In [55]:
target_cpu = target.to('cpu')
input_cpu = input.to('cpu')

def supp_idxs(c):
        # FIXME when torch will support where as np
        return target_cpu.eq(c).nonzero()[:n_support].squeeze(1)
classes = torch.unique(target_cpu)
n_classes = len(classes)
n_query = target_cpu.eq(classes[0].item()).sum().item() - n_support
n_support,n_query

(6, 5)

In [47]:
classes

tensor([2, 3, 5, 6, 7])

In [46]:
target_cpu

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7])

In [69]:
support_idxs = list(map(supp_idxs, classes))
prototypes = torch.stack([input_cpu[idx_list].mean(0) for idx_list in support_idxs])
query_idxs = torch.stack(list(map(lambda c: target_cpu.eq(c).nonzero()[n_support:], classes))).view(-1)
query_samples = input_cpu[query_idxs]
dists = euclidean_dist(query_samples, prototypes)
log_p_y = F.log_softmax(-dists, dim=1).view(n_classes, n_query, -1)

In [71]:
log_p_y.shape

torch.Size([5, 5, 5])

In [68]:
dists,dists.shape

(tensor([[15.8068, 21.4587, 21.0877, 24.9164, 19.2297],
         [16.7870, 55.2084, 53.1533, 61.0909, 50.9115],
         [18.3518, 52.4062, 50.5727, 48.2380, 50.5143],
         [23.2521, 52.3595, 52.1024, 56.9914, 50.2366],
         [15.5227, 33.5283, 33.4234, 33.6671, 31.4605],
         [62.6541,  1.0825,  1.3130,  0.9753,  2.3998],
         [67.7161,  2.9946,  2.5883,  2.1215,  4.3938],
         [54.8156,  0.2482,  1.1325,  2.7638,  0.6256],
         [63.3501,  1.0148,  1.6496,  1.4973,  2.1381],
         [65.3004,  1.5265,  1.8015,  1.4714,  2.7855],
         [46.0608,  1.2502,  1.0670,  1.9801,  1.3890],
         [31.3672, 13.4647, 12.1627, 17.2334, 11.1742],
         [29.0774,  9.9363,  7.9122, 10.4043,  8.5683],
         [52.3547,  1.1000,  1.0389,  2.0864,  0.8667],
         [36.7222,  4.9042,  4.2629,  8.0896,  3.7071],
         [61.7008,  1.7683,  1.9037,  1.9942,  3.1957],
         [55.9874,  3.3078,  2.9891,  5.7719,  3.2949],
         [62.9060, 10.5471,  8.3844,  9.2086, 11

In [20]:
test_df.groupby('label').count()

,data,activity_name,participant
label,,,
0,3,3,3
1,3,3,3
2,3,3,3
3,5,5,5
4,5,5,5
5,5,5,5
6,7,7,7
7,5,5,5


In [23]:
model=model.eval()
data=torch.from_numpy(np.array([d for d in train_df['data']]))
labels=torch.from_numpy(np.array([d for d in train_df['label']]))
data=data.float()  
data=data.to(device)
labels=labels.to(device)
test_acc=Extract_data.get_acc(model,data,labels)

In [66]:
def get_prototypes(labels,embedding):
    def get_indices(c):
        return labels.eq(c).nonzero().squeeze(1)

    classes = torch.unique(labels)
    idxs = list(map(get_indices, classes))
    prototypes = torch.stack([embedding[idx_list].mean(0) for idx_list in idxs])
    
    return prototypes,idxs,classes

def euclidean_dist(x,y):
    return torch.pow(x-y,2).sum()

In [47]:
def get_acc(model,data,labels,pred_index=1):
    ret=model(data)
    pred=ret[pred_index]
    predmax=torch.max(pred,1)[1]
    iscorrect=(predmax==labels)
    num_correct=torch.sum(iscorrect).item()
    num_total=iscorrect.shape[0]
    acc=num_correct/num_total
    return acc

optimizer.param_groups[0]['lr']=optimizer.param_groups[0]['lr']*discount
model=model.eval()
data=torch.from_numpy(np.array([d for d in test_df['data']]))
labels=torch.from_numpy(np.array([d for d in test_df['label']]))
data=data.float()  
data=data.to(device)
labels=labels.to(device)
get_acc(model,data,labels,pred_index=0)

0.0

In [ ]:
selected_batch,_=Extract_data.divide_data_samples(selected_df,num_samples=n_samples)
selected_data=torch.from_numpy(np.array([d for d in selected_batch['data']]))
selected_labels=torch.from_numpy(np.array([d for d in selected_batch['label']]))
selected_data=selected_data.float()  
selected_data=selected_data.to(device)
selected_labels=selected_labels.to(device)

selected_embedding,selected_pred=model(selected_data)

loss, acc,t=prototypical_loss.prototypical_loss(selected_embedding,target=selected_labels,n_support=n_support)

In [62]:
selected_embedding.shape,selected_labels.shape

(torch.Size([55, 128]), torch.Size([55]))

In [63]:
embedding,_=model(data)
embedding.shape,labels.shape

(torch.Size([36, 128]), torch.Size([36]))

In [99]:
torch.cuda.empty_cache()

In [3]:
data=torch.from_numpy(np.array([d for d in test_df['data']]))
labels=torch.from_numpy(np.array([d for d in test_df['label']]))
data=data.float()  
data=data.to(device)
train_labels=labels.to(device)
train_embedding,_=model(data)

In [71]:
n_support=10
selected_df=train_df.groupby('label').head(n_support)
selected_df['from']='train'

min_num=np.min(test_df.groupby('label').count()['data'].values)
df=test_df.groupby('label').head(min_num)
total_df=selected_df.append(df)

data=torch.from_numpy(np.array([d for d in total_df['data']]))
labels=torch.from_numpy(np.array([d for d in total_df['label']]))
data=data.float()  
data=data.to(device)
labels=labels.to(device)
embedding,_=model(data)

loss, acc, t=prototypical_loss.prototypical_loss(embedding,target=labels,n_support=n_support)
per_class_acc=torch.mean(t.float(),dim=1)

source_acc=[str(acc.item()) for acc in per_class_acc]
source_acc=','.join(source_acc)+','
source_acc

'1.0,0.3333333432674408,1.0,1.0,1.0,0.6666666865348816,0.3333333432674408,0.6666666865348816,'

In [38]:
selected_df.groupby('label').count()

,activity_name,data,participant,from
label,,,,
0,10,10,10,10
1,10,10,10,10
2,10,10,10,10
3,10,10,10,10
4,10,10,10,10
5,10,10,10,10
6,10,10,10,10
7,10,10,10,10


In [37]:
df['from']='test'

In [40]:
total_df=selected_df.append(df)

In [42]:
total_df[total_df['label']==0]

,activity_name,data,from,label,participant
1154,walk,"[[5.5069, 6.7566, 7.822, 8.0047, 8.0361, 7.967...",train,0,1
1100,walk,"[[6.9098, 6.8604, 6.3656, 6.3132, 6.1153, 4.94...",train,0,1
828,walk,"[[-0.32388, 0.029639, 0.21350999999999998, 0.4...",train,0,1
316,walk,"[[3.9958, 4.6419, 5.3732, 5.9856, 6.1365, 5.93...",train,0,1
1406,walk,"[[4.9094, 4.5718, 4.4677, 4.695, 4.6732, 4.167...",train,0,1
1587,walk,"[[4.8129, 4.1121, 4.3335, 4.5836, 4.4844, 3.06...",train,0,1
1467,walk,"[[-4.2818, -4.6368, -4.9924, -4.8207, -4.6677,...",train,0,1
186,walk,"[[-1.7091, -1.2767, -0.60736, -0.25776, -0.084...",train,0,1
710,walk,"[[0.42508, 1.0843, 1.568, 1.9041, 2.5528, 2.68...",train,0,1
1537,walk,"[[2.3542, 1.3295, 0.5518, -0.44569, -0.8194799...",train,0,1


In [22]:
optimizer.param_groups[0]['lr']=optimizer.param_groups[0]['lr']*discount
model=model.eval()

df=test_df.groupby('label').head(3)
data=torch.from_numpy(np.array([d for d in df['data']]))
labels=torch.from_numpy(np.array([d for d in df['label']]))
data=data.float()  
data=data.to(device)
labels=labels.to(device)
embedding,_=model(data)

In [23]:
loss, acc,_=prototypical_loss.prototypical_loss(embedding,target=labels,n_support=2)
acc

tensor(0.1250)

In [88]:
target_df.groupby('label').head()

,data,activity_name,participant
label,,,
0,1790,1790,1790
1,1790,1790,1790
2,1790,1790,1790
3,1790,1790,1790
4,1792,1792,1792


In [3]:
torch.cuda.empty_cache()

In [6]:
model=model.to(device)

In [12]:
'''
Look here!!
'''

target_df=target_df.sample(n=len(target_df))
min_num=np.min(target_df.groupby('label').count()['data'].values)
df=target_df.groupby('label').head(min_num)
data=torch.from_numpy(np.array([d for d in df['data']]))
labels=torch.from_numpy(np.array([d for d in df['label']]))
data=data.float()  
data=data.to(device)
labels=labels.to(device)

FSL_embeddings,_=best_model(data)

loss, acc, t=prototypical_loss.prototypical_loss(FSL_embeddings,target=labels,n_support=10)
per_class_acc=torch.mean(t.float(),dim=1)

In [12]:

test_acc.item()

0.625

In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
torch.cuda.empty_cache()

data_df,seen_classes,unseen_classes,sr=Extract_data.prepare_UTWENTE_data()
participants=data_df['participant'].unique()

source_data,target_data=Extract_data.get_seen_unseen_data(data_df,seen_classes,unseen_classes,['1'],['1'])

bs=60
num_features=128
in_channels=data_df['data'].iloc[0].shape[0]
num_classes=len(np.unique(source_data['label']))
lr=0.001
discount=0.6
segment_len_s=1
window_len=sr*segment_len_s
overlap=int(window_len*0.5)
num_selected=5
kernel1_size=2
kernel2_size=1
n_support=6
n_query=5
n_samples=n_support+n_query

file='/home/lnw8px/models/HAR_FSL/UTWENTE/proto/test.txt'
f = open(file, "a") 
source_classes=source_data.groupby('label').head(1).sort_values(by=['label'])['activity_name'].values
target_classes=target_data.groupby('label').head(1).sort_values(by=['label'])['activity_name'].values

source_classes=','.join(source_classes)
target_classes=','.join(target_classes)

towrite='bs,discount,kernel1_size,kernel2_size,participant1,participant2,test_acc,n_shots,'
towrite+='fsl_acc \n'
f.write(towrite)
f.close()
#print(towrite)

#get windowed data
windowed_df=Extract_data.get_windowed_df(data_df,window_len,overlap)
participant1=participant2=participants[0]

for i in range(1000):
    #for participant1 in participants:
        #for participant2 in participants:
    torch.cuda.empty_cache()

    source_df,target_df=Extract_data.get_seen_unseen_data(windowed_df,seen_classes,unseen_classes,[participant1],[participant2])

    min_num=np.min(source_df.groupby('label').count()['data'].values)
    train_df,test_df=Extract_data.divide_data_samples(source_df,num_samples=min_num-3)

    model=FCNN_serial.HARmodel(num_classes=num_classes,in_channels=in_channels,
                        num_features=num_features,
                        kernel1_size=kernel1_size,
                        kernel2_size=kernel2_size)
    model=model.to(device)
    optimizer=torch.optim.Adam(model.parameters(), lr=lr)

    train_loss_list,center_loss_list=[],[]
    max_acc=0
    best_model=0
    for i in range(1000):
        selected_df,_=Extract_data.select_random_classes_data(train_df,num_selected)

        model.zero_grad()
        optimizer.zero_grad()

        selected_batch,_=Extract_data.divide_data_samples(selected_df,num_samples=n_samples)
        selected_data=torch.from_numpy(np.array([d for d in selected_batch['data']]))
        selected_labels=torch.from_numpy(np.array([d for d in selected_batch['label']]))
        selected_data=selected_data.float()  
        selected_data=selected_data.to(device)
        selected_labels=selected_labels.to(device)

        selected_embedding,selected_pred=model(selected_data)

        loss, acc,_=prototypical_loss.prototypical_loss(selected_embedding,target=selected_labels,n_support=n_support)

        loss.backward()
        optimizer.step()

        #train_loss_list.append(label_loss.item())
        #center_loss_list.append(loss_center.item())

        if((i>0) and (i%100==0)):
            optimizer.param_groups[0]['lr']=optimizer.param_groups[0]['lr']*discount
            model=model.eval()
            
            #get per class accuracy on test source data
            n_support=10
            selected_df=train_df.groupby('label').head(n_support)
            selected_df['from']='train'

            min_num=np.min(test_df.groupby('label').count()['data'].values)
            df=test_df.groupby('label').head(min_num)
            total_df=selected_df.append(df)

            data=torch.from_numpy(np.array([d for d in total_df['data']]))
            labels=torch.from_numpy(np.array([d for d in total_df['label']]))
            data=data.float()  
            data=data.to(device)
            labels=labels.to(device)
            embedding,_=model(data)

            loss, test_acc, t=prototypical_loss.prototypical_loss(embedding,target=labels,n_support=n_support)
            test_acc=test_acc.item()
            per_class_source_acc=torch.mean(t.float(),dim=1)
            
            model=model.train()
            if(test_acc>max_acc):
                max_acc=test_acc
                best_model=model

    source_acc=[str(acc.item()) for acc in per_class_source_acc]
    source_acc=','.join(source_acc)+','

    target_df=target_df.sample(n=len(target_df))
    for n_shots in range(1,6):
        min_num=np.min(target_df.groupby('label').count()['data'].values)
        df=target_df.groupby('label').head(min_num)
        data=torch.from_numpy(np.array([d for d in df['data']]))
        labels=torch.from_numpy(np.array([d for d in df['label']]))
        data=data.float()  
        data=data.to(device)
        labels=labels.to(device)

        FSL_embeddings,_=best_model(data)

        loss, acc, t=prototypical_loss.prototypical_loss(FSL_embeddings,target=labels,n_support=n_shots)
        per_class_acc=torch.mean(t.float(),dim=1)
        
        towrite=str(bs)+','+str(discount)+','+str(kernel1_size)+','+str(kernel2_size)+','
        towrite+=str(participant1)+','+str(participant2)+','
        towrite+=str(max_acc)+','
        towrite+=str(n_shots)+','
        towrite+=str(acc.item())+'\n'

        f = open(file, "a")
        f.write(towrite)
        f.close() 

KeyboardInterrupt: 

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
torch.cuda.empty_cache()

data_df,seen_classes,unseen_classes,sr=Extract_data.prepare_UTWENTE_data()
participants=data_df['participant'].unique()

source_data,target_data=Extract_data.get_seen_unseen_data(data_df,seen_classes,unseen_classes,['1'],['1'])

bs=60
num_features=128
in_channels=data_df['data'].iloc[0].shape[0]
num_classes=len(np.unique(source_data['label']))
lr=0.001
discount=0.6
segment_len_s=1
window_len=sr*segment_len_s
overlap=int(window_len*0.5)
num_selected=5
kernel1_size=2
kernel2_size=1
n_support=6
n_query=5
n_samples=n_support+n_query

file='/home/lnw8px/models/HAR_FSL/UTWENTE/proto/test.txt'
f = open(file, "a") 
source_classes=source_data.groupby('label').head(1).sort_values(by=['label'])['activity_name'].values
target_classes=target_data.groupby('label').head(1).sort_values(by=['label'])['activity_name'].values

source_classes=','.join(source_classes)
target_classes=','.join(target_classes)

towrite='bs,discount,kernel1_size,kernel2_size,participant1,participant2,test_acc,n_shots,'
towrite+='fsl_acc'
f.write(towrite)
f.close()
#print(towrite)

#get windowed data
windowed_df=Extract_data.get_windowed_df(data_df,window_len,overlap)
participant1=participant2=participants[0]

for i in range(1000):
    #for participant1 in participants:
        #for participant2 in participants:
    torch.cuda.empty_cache()

    source_df,target_df=Extract_data.get_seen_unseen_data(windowed_df,seen_classes,unseen_classes,[participant1],[participant2])

    min_num=np.min(source_df.groupby('label').count()['data'].values)
    train_df,test_df=Extract_data.divide_data_samples(source_df,num_samples=min_num-3)

    model=FCNN_serial.HARmodel(num_classes=num_classes,in_channels=in_channels,
                        num_features=num_features,
                        kernel1_size=kernel1_size,
                        kernel2_size=kernel2_size)
    model=model.to(device)
    optimizer=torch.optim.Adam(model.parameters(), lr=lr)

    train_loss_list,center_loss_list=[],[]
    max_acc=0
    best_model=0
    for i in range(1000):
        selected_df,_=Extract_data.select_random_classes_data(train_df,num_selected)

        model.zero_grad()
        optimizer.zero_grad()

        selected_batch,_=Extract_data.divide_data_samples(selected_df,num_samples=n_samples)
        selected_data=torch.from_numpy(np.array([d for d in selected_batch['data']]))
        selected_labels=torch.from_numpy(np.array([d for d in selected_batch['label']]))
        selected_data=selected_data.float()  
        selected_data=selected_data.to(device)
        selected_labels=selected_labels.to(device)

        selected_embedding,selected_pred=model(selected_data)

        loss, acc,_=prototypical_loss.prototypical_loss(selected_embedding,target=selected_labels,n_support=n_support)

        loss.backward()
        optimizer.step()

        #train_loss_list.append(label_loss.item())
        #center_loss_list.append(loss_center.item())

        if((i>0) and (i%100==0)):
            optimizer.param_groups[0]['lr']=optimizer.param_groups[0]['lr']*discount
            model=model.eval()
            data=torch.from_numpy(np.array([d for d in test_df['data']]))
            labels=torch.from_numpy(np.array([d for d in test_df['label']]))
            data=data.float()  
            data=data.to(device)
            labels=labels.to(device)
            test_acc=Extract_data.get_acc(model,data,labels)
            model=model.train()
            if(test_acc>max_acc):
                max_acc=test_acc
                best_model=model

    best_model=best_model.eval()
    #get per class accuracy on test source data
    n_support=10
    selected_df=train_df.groupby('label').head(n_support)
    selected_df['from']='train'

    min_num=np.min(test_df.groupby('label').count()['data'].values)
    df=test_df.groupby('label').head(min_num)
    total_df=selected_df.append(df)

    data=torch.from_numpy(np.array([d for d in total_df['data']]))
    labels=torch.from_numpy(np.array([d for d in total_df['label']]))
    data=data.float()  
    data=data.to(device)
    labels=labels.to(device)
    embedding,_=model(data)

    loss, acc, t=prototypical_loss.prototypical_loss(embedding,target=labels,n_support=n_support)
    per_class_acc=torch.mean(t.float(),dim=1)

    source_acc=[str(acc.item()) for acc in per_class_acc]
    source_acc=','.join(source_acc)+','
    source_acc

    target_df=target_df.sample(n=len(target_df))
    for n_shots in range(1,6):
        min_num=np.min(target_df.groupby('label').count()['data'].values)
        df=target_df.groupby('label').head(min_num)
        data=torch.from_numpy(np.array([d for d in df['data']]))
        labels=torch.from_numpy(np.array([d for d in df['label']]))
        data=data.float()  
        data=data.to(device)
        labels=labels.to(device)

        FSL_embeddings,_=best_model(data)

        loss, acc, t=prototypical_loss.prototypical_loss(FSL_embeddings,target=labels,n_support=n_shots)
        per_class_acc=torch.mean(t.float(),dim=1)
        
        towrite=str(bs)+','+str(discount)+','+str(kernel1_size)+','+str(kernel2_size)+','
        towrite+=str(participant1)+','+str(participant2)+','
        towrite+=str(max_acc)+','
        towrite+=str(n_shots)+','
        towrite+=str(acc)

        f = open(file, "a")
        f.write(towrite)
        f.close() 

KeyboardInterrupt: 

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
torch.cuda.empty_cache()

data_df,seen_classes,unseen_classes,sr=Extract_data.prepare_UTWENTE_data()
participants=data_df['participant'].unique()

source_data,target_data=Extract_data.get_seen_unseen_data(data_df,seen_classes,unseen_classes,['1'],['1'])

bs=60
num_features=128
in_channels=data_df['data'].iloc[0].shape[0]
num_classes=len(np.unique(source_data['label']))
lr=0.001
discount=0.6
segment_len_s=1
window_len=sr*segment_len_s
overlap=int(window_len*0.5)
num_selected=5
kernel1_size=2
kernel2_size=1
n_support=6
n_query=5
n_samples=n_support+n_query

file='/home/lnw8px/models/HAR_FSL/test.txt'
f = open(file, "a") 
source_classes=source_data.groupby('label').head(1).sort_values(by=['label'])['activity_name'].values
target_classes=target_data.groupby('label').head(1).sort_values(by=['label'])['activity_name'].values

source_classes=','.join(source_classes)
target_classes=','.join(target_classes)

towrite='bs,discount,kernel1_size,kernel2_size,participant1,participant2,test_acc,n_shots,'
towrite+='sim_acc,svm_acc,knn_total_acc,'
towrite+=source_classes+','
towrite+=','.join(['sim '+item for item in target_classes.split(',')])+','
towrite+=','.join(['svm '+item for item in target_classes.split(',')])+','
towrite+=','.join(['knn '+item for item in target_classes.split(',')])+','
towrite+='len_source,len_target,sim_source,sim_target,svm_source,svm_target,knn_source,knn_target\n'
f.write(towrite)
f.close()
#print(towrite)

#get windowed data
windowed_df=Extract_data.get_windowed_df(data_df,window_len,overlap)
participant1=participant2=participants[0]

for i in range(1):
    #for participant1 in participants:
        #for participant2 in participants:
    torch.cuda.empty_cache()

    source_df,target_df=Extract_data.get_seen_unseen_data(windowed_df,seen_classes,unseen_classes,[participant1],[participant2])

    min_num=np.min(source_df.groupby('label').count()['data'].values)
    train_df,test_df=Extract_data.divide_data_samples(source_df,num_samples=min_num-3)
    #train_df,test_df=Extract_data.divide_data_ratio(source_data_df,0.8)

    #calculate weights for classes
    counts=train_df.groupby(['label']).count()['data'].values
    weights=[1/item for item in counts]
    weights=torch.tensor(weights/np.sum(weights))
    weights=weights.float()
    weights=weights.to(device)

    model=FCNN_serial.HARmodel(num_classes=num_classes,in_channels=in_channels,
                        num_features=num_features,
                        kernel1_size=kernel1_size,
                        kernel2_size=kernel2_size)
    model=model.to(device)
    optimizer=torch.optim.Adam(model.parameters(), lr=lr)

    train_loss_list,center_loss_list=[],[]
    max_acc=0
    best_model=0
    for i in range(1000):
        selected_df,_=Extract_data.select_random_classes_data(train_df,num_selected)

        model.zero_grad()
        optimizer.zero_grad()

        selected_batch,_=Extract_data.divide_data_samples(selected_df,num_samples=n_samples)
        selected_data=torch.from_numpy(np.array([d for d in selected_batch['data']]))
        selected_labels=torch.from_numpy(np.array([d for d in selected_batch['label']]))
        selected_data=selected_data.float()  
        selected_data=selected_data.to(device)
        selected_labels=selected_labels.to(device)

        selected_embedding,selected_pred=model(selected_data)

        loss, acc,_=prototypical_loss.prototypical_loss(selected_embedding,target=selected_labels,n_support=n_support)

        loss.backward()
        optimizer.step()

        #train_loss_list.append(label_loss.item())
        #center_loss_list.append(loss_center.item())

        if((i>0) and (i%100==0)):
            optimizer.param_groups[0]['lr']=optimizer.param_groups[0]['lr']*discount
            model=model.eval()
            data=torch.from_numpy(np.array([d for d in test_df['data']]))
            labels=torch.from_numpy(np.array([d for d in test_df['label']]))
            data=data.float()  
            data=data.to(device)
            labels=labels.to(device)
            test_acc=Extract_data.get_acc(model,data,labels)
            model=model.train()
            if(test_acc>max_acc):
                max_acc=test_acc
                best_model=model

    best_model=best_model.eval()
    #get per class accuracy on test source data
    n_support=10
    selected_df=train_df.groupby('label').head(n_support)
    selected_df['from']='train'

    min_num=np.min(test_df.groupby('label').count()['data'].values)
    df=test_df.groupby('label').head(min_num)
    total_df=selected_df.append(df)

    data=torch.from_numpy(np.array([d for d in total_df['data']]))
    labels=torch.from_numpy(np.array([d for d in total_df['label']]))
    data=data.float()  
    data=data.to(device)
    labels=labels.to(device)
    embedding,_=model(data)

    loss, acc, t=prototypical_loss.prototypical_loss(embedding,target=labels,n_support=n_support)
    per_class_acc=torch.mean(t.float(),dim=1)

    source_acc=[str(acc.item()) for acc in per_class_acc]
    source_acc=','.join(source_acc)+','
    source_acc

    for j in range(1):
        target_df=target_df.sample(n=len(target_df))
        for n_shots in range(1,6):
            #extract data from FSL
            FSL_train_df=target_df.groupby(['label']).head(n_shots)
            train_indices=FSL_train_df.index
            FSL_test_df=target_df[~target_df.index.isin(train_indices)]

            #extract same number of samples from both labels
            min_samples=min(FSL_test_df.groupby('label').count().iloc[:,0].values)
            FSL_test_df=FSL_test_df.groupby('label').head(min_samples)

            FSL_train_data=torch.from_numpy(np.array([d for d in FSL_train_df['data']]))
            FSL_train_labels=torch.from_numpy(np.array([d for d in FSL_train_df['label']]))
            FSL_train_data=FSL_train_data.float()  

            FSL_test_data=torch.from_numpy(np.array([d for d in FSL_test_df['data']]))
            FSL_test_labels=torch.from_numpy(np.array([d for d in FSL_test_df['label']]))
            FSL_test_data=FSL_test_data.float()  

            FSL_train_data=FSL_train_data.to(device)
            FSL_test_data=FSL_test_data.to(device)

            FSL_train_embeddings,_=best_model(FSL_train_data)
            FSL_test_embeddings,_=best_model(FSL_test_data)

            FSL_train_df=pd.DataFrame()
            FSL_train_df['data']=list(FSL_train_embeddings.cpu().detach().numpy())
            FSL_train_df['label']=list(FSL_train_labels.cpu().detach().numpy())

            FSL_test_df=pd.DataFrame()
            FSL_test_df['data']=list(FSL_test_embeddings.cpu().detach().numpy())
            FSL_test_df['label']=list(FSL_test_labels.cpu().detach().numpy())

            sim_total_acc,sim_per_class_acc=Inference.train_similarity_classifier(FSL_train_df.copy(deep=True),FSL_test_df.copy(deep=True))
            svm_total_acc,svm_per_class_acc=Inference.train_SVM(FSL_train_df.copy(deep=True),FSL_test_df.copy(deep=True))
            knn_total_acc,_,_,knn_per_class_acc=Inference.train_KNN(FSL_train_df.copy(deep=True),FSL_test_df.copy(deep=True))
            towrite=str(bs)+','+str(discount)+','+str(kernel1_size)+','+str(kernel2_size)+','
            towrite+=str(participant1)+','+str(participant2)+','
            towrite+=str(max_acc)+','
            towrite+=str(n_shots)+','
            towrite+=str(sim_total_acc)+','
            towrite+=str(svm_total_acc)+','
            towrite+=str(knn_total_acc)+','
            towrite+=source_acc
            sim_per_class_acc=list(sim_per_class_acc['iscorrect'].values)
            sim_per_class_acc=','.join([str(item) for item in sim_per_class_acc])
            svm_per_class_acc=list(svm_per_class_acc['iscorrect'].values)
            svm_per_class_acc=','.join([str(item) for item in svm_per_class_acc])
            knn_per_class_acc=list(knn_per_class_acc['iscorrect'].values)
            knn_per_class_acc=','.join([str(item) for item in knn_per_class_acc])

            towrite+=sim_per_class_acc+','
            towrite+=svm_per_class_acc+','
            towrite+=knn_per_class_acc+','

            #classify source vs target data
            test_df_cp=test_df.copy(deep=True)
            target_df_cp=target_df[['data','label']].copy(deep=True)
            test_df_cp['source']=1
            target_df_cp['source']=0
            total_df=pd.DataFrame()
            total_df=target_df_cp.append(test_df_cp)
            selected_train_df=total_df.groupby(['label','source']).head(n_shots)
            train_indices=selected_train_df.index
            selected_test_df=total_df[~total_df.index.isin(train_indices)]
            selected_train_df.groupby(['label','source']).count(),selected_test_df.groupby(['label','source']).count()
            selected_train_df['label']=selected_train_df['source']
            selected_test_df['label']=selected_test_df['source']

            selected_train_data=torch.from_numpy(np.array([d for d in selected_train_df['data']]))
            selected_train_labels=np.array([d for d in selected_train_df['label']])
            selected_train_data=selected_train_data.float()  
            selected_train_data=selected_train_data.to(device)
            selected_train_embedding,_=model(selected_train_data)
            selected_train_embedding=selected_train_embedding.cpu().detach().numpy()

            selected_test_data=torch.from_numpy(np.array([d for d in selected_test_df['data']]))
            selected_test_labels=np.array([d for d in selected_test_df['label']])
            selected_test_data=selected_test_data.float()  
            selected_test_data=selected_test_data.to(device)
            selected_test_embedding,_=model(selected_test_data)
            selected_test_embedding=selected_test_embedding.cpu().detach().numpy()

            emb_train_df=pd.DataFrame()
            emb_train_df['data']=list(selected_train_embedding)
            emb_train_df['label']=selected_train_labels

            emb_test_df=pd.DataFrame()
            emb_test_df['data']=list(selected_test_embedding)
            emb_test_df['label']=selected_test_labels

            #extract same number of samples from both labels
            min_samples=min(emb_test_df.groupby('label').count().values)[0]
            emb_test_df=emb_test_df.groupby('label').head(min_samples)

            sim_total_acc,sim_per_class_acc=Inference.train_similarity_classifier(emb_train_df.copy(deep=True),emb_test_df.copy(deep=True))
            knn_total_acc,pred,test_labels,knn_per_class_acc=Inference.train_KNN(emb_train_df.copy(deep=True),emb_test_df.copy(deep=True))
            svm_total_acc,svm_per_class_acc=Inference.train_SVM(emb_train_df.copy(deep=True),emb_test_df.copy(deep=True))

            counts=emb_test_df.groupby('label').count()['data'].values
            towrite+=str(counts[0])+','+str(counts[1])+','
            sim_per_class_acc=list(sim_per_class_acc['iscorrect'].values)
            knn_per_class_acc=list(knn_per_class_acc['iscorrect'].values)
            svm_per_class_acc=list(svm_per_class_acc['iscorrect'].values)
            towrite+=str(sim_per_class_acc[0])+','+str(sim_per_class_acc[1])+','
            towrite+=str(svm_per_class_acc[0])+','+str(svm_per_class_acc[1])+','
            towrite+=str(knn_per_class_acc[0])+','+str(knn_per_class_acc[1])+'\n'

            f = open(file, "a")
            f.write(towrite)
            f.close() 